In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

Wyświetlenie przykładowych zlabelowanych obrazów, które będą użyte do trenowania modelu:

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

images_path = '/content/drive/My Drive/projekt_znaki/train/images'
labels_path = '/content/drive/My Drive/projekt_znaki/train/labels'

def draw_bounding_boxes(image, label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        # yolo na wspolrzedne obrazu
        img_height, img_width = image.shape[:2]
        x_center *= img_width
        y_center *= img_height
        width *= img_width
        height *= img_height

        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image

for image_name in os.listdir(images_path)[1:10]:  # wyswietlenie kilku obrazow
#for image_name in os.listdir(images_path):  # wyswietlenie wszystkich obrazow
    if image_name.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(images_path, image_name)
        label_path = os.path.join(labels_path, os.path.splitext(image_name)[0] + '.txt')

        image = cv2.imread(image_path)
        if image is None:
            print(f"Nie można wczytać obrazu: {image_path}")
            continue
        if not os.path.exists(label_path):
            print(f"Etykieta nie istnieje: {label_path}")
            continue

        image_with_boxes = draw_bounding_boxes(image, label_path)

        print(f"Nazwa obrazu: {image_name}")
        plt.figure(figsize=(10, 5))
        plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        display(plt.gcf())



In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans

def load_yolo_labels(label_path):
    boxes = []
    for filename in os.listdir(label_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(label_path, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    _, x_center, y_center, width, height = map(float, line.strip().split())
                    boxes.append((width, height))
    return np.array(boxes)

label_path = '/content/drive/My Drive/projekt_znaki/train/labels'
boxes = load_yolo_labels(label_path)

print(f"Wymiary bbox: {boxes.shape}")

num_anchors = 9

kmeans = KMeans(n_clusters=num_anchors, random_state=0).fit(boxes)
anchors = kmeans.cluster_centers_

print("Anchor boxy:")
print(anchors)

anchors = anchors * np.array([1200, 900])

anchors = anchors.round().astype(int).tolist()

print("Anchor boxy w pikselach:")
for anchor in anchors:
    print(anchor)


Augmentacja danych - przekształcenie obrazów:

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import albumentations as A

def convert_bbox_to_yolo(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return [x_center, y_center, width, height]

image_dir = '/content/drive/My Drive/projekt_znaki/train/images'
label_dir = '/content/drive/My Drive/projekt_znaki/train/labels'
aug_image_dir = '/content/drive/My Drive/projekt_znaki/train/augmented/images'
aug_label_dir = '/content/drive/My Drive/projekt_znaki/train/augmented/labels'

os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)

transform = A.Compose([
    A.Rotate(limit=45, p=1.0),

    A.RandomBrightnessContrast(p=0.2)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_visibility=0.3))

for image_name in os.listdir(image_dir):
    if image_name.endswith('.png'):
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        img_height, img_width = image.shape[:2]

        label_path = os.path.join(label_dir, image_name.replace('.png', '.txt'))
        with open(label_path, 'r') as file:
            bboxes = []
            class_labels = []
            for line in file.readlines():
                elements = line.split()
                class_labels.append(int(elements[0]))
                bbox_values = list(map(float, elements[1:]))
                x_center, y_center, width, height = bbox_values
                x_min = (x_center - width / 2) * img_width
                y_min = (y_center - height / 2) * img_height
                x_max = (x_center + width / 2) * img_width
                y_max = (y_center + height / 2) * img_height
                bboxes.append([x_min, y_min, x_max, y_max])

        # augmentacja
        for i in range(5):
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_labels = augmented['class_labels']

            aug_image_name = image_name.replace('.png', f'_aug_{i}.png')
            aug_image_path = os.path.join(aug_image_dir, aug_image_name)

            cv2.imwrite(aug_image_path, aug_image)

            aug_label_path = os.path.join(aug_label_dir, aug_image_name.replace('.png', '.txt'))
            with open(aug_label_path, 'w') as file:
                for bbox, label in zip(aug_bboxes, aug_labels):
                    yolo_bbox = convert_bbox_to_yolo(bbox, img_width, img_height)
                    file.write(f"{label} {' '.join(map(str, yolo_bbox))}\n")

images_path = aug_image_dir
labels_path = aug_label_dir

def draw_bounding_boxes(image, label_path, image_name):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        try:
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            img_height, img_width = image.shape[:2]
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height

            x_min = int(x_center - width / 2)
            y_min = int(y_center - height / 2)
            x_max = int(x_center + width / 2)
            y_max = int(y_center + height / 2)

            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, str(class_id), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except ValueError as e:
            print(f"Error processing file {image_name}: {e}")

    return image

def display_images(images, titles, cols=3):
    rows = len(images) // cols + int(len(images) % cols != 0)
    plt.figure(figsize=(20, 10))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

images = []
titles = []
for image_name in os.listdir(images_path)[6:12]:
    if image_name.endswith(('.png', '.jpeg')):
        image_path = os.path.join(images_path, image_name)
        label_path = os.path.join(labels_path, os.path.splitext(image_name)[0] + '.txt')

        image = cv2.imread(image_path)
        image_with_boxes = draw_bounding_boxes(image, label_path, image_name)

        images.append(image_with_boxes)
        titles.append(image_name)

display_images(images, titles)


In [ ]:
!pip install ultralytics

Trenowanie modelu - najlepszy model zapisuje się automatycznie:

In [ ]:
import os
import yaml
from ultralytics import YOLO

with open('/content/drive/My Drive/projekt_znaki/yolov8_config.yaml') as f:
    config = yaml.safe_load(f)

model = YOLO(config['model']['type'])

data = '/content/drive/My Drive/projekt_znaki/yolov8_config.yaml'

val_data = '/content/drive/My Drive/projekt_znaki/val'

model.train(data=data, epochs=50, imgsz=[1200, 900])

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/mgr 1 semestr/ai/projekt_znaki/yolov8_config.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=[1200, 900], save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning /content/drive/My Drive/mgr 1 semestr/ai/projekt_znaki/train/augmented/labels.cache... 615 images, 11 backgrounds, 0 corrupt: 100%|██████████| 615/615 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/mgr 1 semestr/ai/projekt_znaki/val/labels.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000435, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1216 train, 1216 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      7.99G     0.6903      1.094     0.8577         24       1216: 100%|██████████| 39/39 [00:48<00:00,  1.24s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]

                   all         30         84      0.712      0.711      0.781      0.532



50 epochs completed in 0.804 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3009353 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.39it/s]


                   all         30         84      0.714       0.68      0.772      0.542
droga_z_pierwszenstwem          7          8      0.553        0.5      0.527      0.375
                dzieci          1          2      0.518          1      0.663      0.464
       jednokierunkowa          2          2      0.929        0.5      0.586      0.401
  koniec_pierwszenstwa          2          2          1      0.672      0.995      0.697
           nakaz_jazdy          2          2      0.471          1      0.995      0.696
ograniczenie_predkosci          2          2      0.284        0.5      0.347       0.17
   ograniczenie_tonazu          1          1      0.594          1      0.995      0.697
               parking          4          9      0.718      0.444      0.558      0.399
przejscie_dla_pieszych         15         24       0.94      0.653      0.833      0.516
            przystanek          2          2      0.704        0.5      0.828      0.581
                 rond

Sprawdzanie działania modelu na obrazkach (które nie były używane do trenowania):

In [ ]:
from PIL import Image
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import os

model = YOLO('/content/drive/My Drive/projekt_znaki/runs/detect/train2/weights/best.pt')

images_path = '/content/drive/My Drive/projekt_znaki/obrazkiDoTestow'

image_files = [f for f in os.listdir(images_path) if f.endswith('.png')]
print(image_files)

for image_file in image_files:
    image_path = os.path.join(images_path, image_file)

    image = cv2.imread(image_path)

    results = model(image)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()

    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box
        confidence = conf
        detected_class = cls
        name = names[int(cls)]

        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        label = f"{name}: {confidence:.2f}"
        cv2.putText(image, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2_imshow(image)
